In [ ]:


!pip install tensorflow

import numpy as np
import os
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator(rescale=1.0/255)

test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam

base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(5, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
import os


train_dir = '/content/drive/MyDrive/train'
test_dir = '/content/drive/MyDrive/test'

if not os.path.exists(train_dir) or not os.path.exists(test_dir):
    raise FileNotFoundError("❌ Dataset directory not found. Check your file paths.")

def remove_missing_files(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            if not os.path.exists(file_path):
                print(f"🚫 Skipping missing file: {file_path}")
                os.remove(file_path)

remove_missing_files(train_dir)
remove_missing_files(test_dir)


history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples,
    epochs=10,
    validation_data=test_generator,
    validation_steps=test_generator.samples
)

In [ ]:
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

In [ ]:
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.show()

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

img_path = '/content/drive/MyDrive/test/fire/fire1.jpeg'
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0

predictions = model.predict(img_array)
class_names = list(train_generator.class_indices.keys())
predicted_class = class_names[np.argmax(predictions)]

print(f" Predicted Event: {predicted_class}")

In [ ]:
if predicted_class == 'dense_crowd':
    print("ALERT: Dense crowd detected! Take immediate action.")
elif predicted_class == 'moderate_crowd':
    print("Notice: Moderate crowd detected.")
elif predicted_class == 'sparse_crowd':
    print("Sparse crowd detected. All clear.")
elif predicted_class == 'heat_stroke':
    print("ALERT: Heat stroke signs detected! Send medical assistance.")
elif predicted_class == 'fainting':
    print("ALERT: Fainting detected! Provide first aid.")
elif predicted_class == 'stampede':
    print("ALERT: Stampede risk! Take crowd control measures.")
elif predicted_class == 'fire':
    print("ALERT: Fire detected! Deploy firefighting resources immediately.")

In [ ]:
model.save('/content/drive/MyDrive/crowd_fire_model.keras')

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('/content/drive/MyDrive/crowd_fire_model.keras')
print("✅ Model loaded successfully!")


In [ ]:
!ngrok config add-authtoken 2tH860vipU68sBaoNOaniZITgOQ_6cNqevwesNNxVcs2JbnKL


In [ ]:
from pyngrok import ngrok

public_url = ngrok.connect(5000)
print("🌐 Public URL:", public_url)


In [ ]:
from flask import Flask, request, jsonify
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
from pyngrok import ngrok
import os

app = Flask(__name__)


model = load_model('/content/drive/MyDrive/crowd_fire_model.keras')


class_names = ['dense_crowd', 'fainting', 'fire', 'moderate_crowd', 'sparse_crowd']


@app.route("/predict", methods=["POST"])
def predict():
    try:
        if 'image' not in request.files:
            print("❌ No image in request")
            return jsonify({"error": "No image uploaded"}), 400

       
        f = request.files['image']
        temp_path = "temp.jpg"
        f.save(temp_path)

     
        img = image.load_img(temp_path, target_size=(224, 224))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0) / 255.0

      
        predictions = model.predict(img_array)
        label = class_names[np.argmax(predictions)]

        os.remove(temp_path)
        print("✅ Prediction:", label)
        return jsonify({"label": label})

    except Exception as e:
        print("❌ Internal server error:", str(e))
        return jsonify({"error": str(e)}), 500


public_url = ngrok.connect(5000)
print("🌐 Public URL:", public_url)


app.run(port=5000)


In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
import os

# Load the trained model
model = load_model('/Users/reham/saved_models/model.h5')

# Class names (if you don't have train_generator anymore)
class_names = ['dense_crowd', 'fainting', 'fire', 'moderate_crowd', 'sparse_crowd']

# Path to test images
test_images_dir = 'downloads/test/'

# Collect image paths
image_paths = []
for class_name in os.listdir(test_images_dir):
    class_folder = os.path.join(test_images_dir, class_name)
    if os.path.isdir(class_folder):
        for filename in os.listdir(class_folder):
            if filename.endswith(('.png', '.jpg', '.jpeg')):
                image_paths.append(os.path.join(class_folder, filename))

# Predict each image
for img_path in image_paths:
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    predictions = model.predict(img_array)
    predicted_class = class_names[np.argmax(predictions)]

    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Predicted Event: {predicted_class}")
    plt.show()

    print(f"Image: {img_path} | Predicted Event: {predicted_class}")


In [ ]:
# Data Augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    'downloads/train',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    'downloads/test',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')  # 7 classes for dense crowd, moderate crowd, sparse crowd, heat stroke, fainting, stampede, and fire
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
# Ensure dataset directories exist
if not os.path.exists ('downloads/train') or not os.path.exists('downloads/test'):
    raise FileNotFoundError("Dataset directory not found. Check your file paths.")

# Remove missing files before training
def remove_missing_files(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            if not os.path.exists('downloads/train'):
                print(f"Skipping missing file: {'downloads/train'}")
                os.remove('downloads/train')  # Remove broken file reference

remove_missing_files('downloads/train')
remove_missing_files ('downloads/test')

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size
)


In [ ]:
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

In [ ]:
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.show()

In [ ]:
img_path = 'downloads/test/fire/fire1.jpeg'
img = image.load_img(img_path, target_size=(150, 150))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0

predictions = model.predict(img_array)
class_names = list(train_generator.class_indices.keys())
predicted_class = class_names[np.argmax(predictions)]

print(f"Predicted Event: {predicted_class}")

In [ ]:
class_weights = {0: 1.0, 1: 2.0, 2: 3.0, 3: 3.0, 4: 4.0, 5: 5.0}
model.fit(train_generator, epochs=10, class_weight=class_weights)

In [ ]:
img_path = 'downloads/test/fire/fire1.jpeg'
img = image.load_img(img_path, target_size=(150, 150))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0

predictions = model.predict(img_array)
class_names = list(train_generator.class_indices.keys())
predicted_class = class_names[np.argmax(predictions)]

print(f"Predicted Event: {predicted_class}")

In [ ]:
if predicted_class == 'dense_crowd':
    print("ALERT: Dense crowd detected! Take immediate action.")
elif predicted_class == 'moderate_crowd':
    print("Notice: Moderate crowd detected.")
elif predicted_class == 'sparse_crowd':
    print("Sparse crowd detected. All clear.")
elif predicted_class == 'heat_stroke':
    print("ALERT: Heat stroke signs detected! Send medical assistance.")
elif predicted_class == 'fainting':
    print("ALERT: Fainting detected! Provide first aid.")
elif predicted_class == 'stampede':
    print("ALERT: Stampede risk! Take crowd control measures.")
elif predicted_class == 'fire':
    print("ALERT: Fire detected! Deploy firefighting resources immediately.")

In [ ]:
from tensorflow.keras.models import load_model
model.save('/Users/reham/saved_models/model.h5')

# Load your trained model (replace with your actual model path)
model = load_model('/Users/reham/saved_models/model.h5')

# Directory containing the test images (adjust path as needed)
test_images_dir = 'downloads/test/'

# Get all image file paths in the test_images_dir
image_paths = []
for class_name in os.listdir(test_images_dir):
    class_folder = os.path.join(test_images_dir, class_name)
    if os.path.isdir(class_folder):  # Only include folders
        for filename in os.listdir(class_folder):
            if filename.endswith(('.png', '.jpg', '.jpeg')):  # Check for image files
                image_paths.append(os.path.join(class_folder, filename))

# Loop through each image, load, predict, and display the result
for img_path in image_paths:
    # Load image and resize to match model input
    img = image.load_img(img_path, target_size=(150, 150))

    # Convert the image to an array and normalize
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0  # Normalize image

    # Make prediction
    predictions = model.predict(img_array)

    # Get the class names from your generator
    class_names = list(train_generator.class_indices.keys())

    # Get the predicted class
    predicted_class = class_names[np.argmax(predictions)]

    # Display the image and the prediction
    plt.imshow(img)
    plt.axis('off')  # Turn off axis labels
    plt.title(f"Predicted Event: {predicted_class}")
    plt.show()

    print(f"Image: {img_path} | Predicted Event: {predicted_class}")

In [ ]:
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0  # Ensure normalization

In [ ]:
class_names = list(train_generator.class_indices.keys())  # Get class labels
predicted_class_name = class_names[predicted_class]
print(f"Predicted Event: {predicted_class_name}")


In [ ]:
img_path = 'downloads/test/fainting/fait1.jpeg'
img = image.load_img(img_path, target_size=(150, 150))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0

predictions = model.predict(img_array)
class_names = list(train_generator.class_indices.keys())
predicted_class = class_names[np.argmax(predictions)]

print(f"Predicted Event: {predicted_class}")

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Get the true labels and predicted labels
y_true = test_generator.classes  # Actual labels
y_pred = model.predict(test_generator)  # Predicted labels
y_pred = np.argmax(y_pred, axis=1)  # Convert predictions to class indices

# Create the confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=train_generator.class_indices.keys(), yticklabels=train_generator.class_indices.keys())
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
import os
import numpy as np

# Ensure files exist before prediction
def safe_predict(generator):
    for batch_images, batch_labels in generator:
        valid_images = []

        # Check if each image in the batch exists
        for image in batch_images:
            image_path = image  # Assuming image path can be extracted from the image (you may need to adjust this)

            if not os.path.exists(image_path):
                print(f"Skipping missing file: {image_path}")
                continue  # Skip this image if missing

            valid_images.append(image)  # Add valid image to the list

        # Yield only valid images in the batch
        yield np.array(valid_images), batch_labels

# Use safe_predict to generate predictions
y_pred = model.predict(safe_predict(test_generator))


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data augmentation setup
train_datagen = ImageDataGenerator(
    rescale=1.0/255,  # Normalize pixel values
    rotation_range=30,  # Random rotation up to 30 degrees
    width_shift_range=0.2,  # Horizontal shift
    height_shift_range=0.2,  # Vertical shift
    shear_range=0.2,  # Shearing transformation
    zoom_range=0.2,  # Random zoom
    horizontal_flip=True,  # Flip images horizontally
    fill_mode='nearest'  # Fill missing pixels
)

# Test data should NOT have augmentation, only normalization
test_datagen = ImageDataGenerator(rescale=1.0/255)

# Load augmented training data from directory
train_generator = train_datagen.flow_from_directory(
    'downloads/train',  # Replace with your training dataset path
    target_size=(150, 150),  # Resize images to this size
    batch_size=32,
    class_mode='categorical'  # Use 'binary' for two classes
)

# Load test data
test_generator = test_datagen.flow_from_directory(
    'downloads/test',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model with augmented data
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator,
    steps_per_epoch=len(train_generator),
    validation_steps=len(test_generator)
)


In [ ]:
import matplotlib.pyplot as plt

augmented_images, _ = next(train_generator)

plt.figure(figsize=(10, 10))
for i in range(5):
    plt.subplot(1, 5, i + 1)
    plt.imshow(augmented_images[i])
    plt.axis('off')
plt.show()


In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Save the model in the recommended format
model.save('augmented_model.keras')
